In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-v3'
device = 'cuda'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [8]:
data = pd.read_csv('./data.csv',encoding='latin-1')[:5000]

In [9]:
data

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
4995,8794,53746,"Nagpur, India",18-03-2020,Why is Government not transmitting benefits of...,Positive
4996,8795,53747,London,18-03-2020,"""As long as we're not seeing markets I would c...",Extremely Positive
4997,8796,53748,"Victoria, London",18-03-2020,Will school fees be refunded if the #coronavir...,Neutral
4998,8797,53749,United States,18-03-2020,"#USD continues its dominance, markets rebounde...",Negative


In [10]:
X = data['OriginalTweet']
y = data['Sentiment']

In [11]:
words = []
data = []
idx = 0
labels = {}
labels_r = {}

In [12]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

5000it [00:03, 1580.99it/s]


In [14]:
words = sorted(set(words))
np.random.shuffle(words)

In [15]:
np.random.shuffle(data)

In [16]:
X = []
y = []

In [17]:
for X_batch,y_batch in tqdm(data):
    X.append(bag_of_words(X_batch,words))
    y.append(y_batch)

100%|██████████████████████████████████████| 5000/5000 [00:23<00:00, 210.17it/s]


In [18]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [19]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [20]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [21]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.iters = 2
        self.dropout = Dropout()
        self.hidden = 512
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,self.hidden)
        self.linear4 = Linear(self.hidden,self.hidden)
        self.linear5 = Linear(self.hidden,self.hidden)
        self.output = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        for _ in range(self.iters):
            preds = self.dropout(self.activation(self.linear3(preds)))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [22]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [23]:
torch.save(model,'model-custom.pt')
torch.save(model,'model-custom.pth')
torch.save(model.state_dict(),'model-custom-sd.pt')
torch.save(model.state_dict(),'model-custom-sd.pth')
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')

In [24]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


100%|█████████████████████████████████████████| 100/100 [01:10<00:00,  1.43it/s]


Loss,0.00395
_runtime,77
_timestamp,1633792255
_step,399
Val Loss,0.20197
Acc,99.5
Val Acc,45.6


Loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Val Loss,▁▂▆▅▅▆▇▆▆▇▇▇▇▇▇█▇▇█▇█▇▇▇▇█▇▇▇▇█▆▇▇█▇▇▇▇▇
Acc,▁▆▇▇████████████████████████████████████
Val Acc,▂▃▆██▅▅▅▇▅▄▅▅▅▆▁▅▆▄▅▃▆▄▆▆▄▆▆▅▄▅▇▅▇▃▇█▇▇▆


In [25]:
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')